In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.classical import expr

from qiskit.transpiler import PassManager
from LogicalQ.Transpilation.DecomposeIfElseOps import DecomposeIfElseOpsTask

from pytket.extensions.qiskit import qiskit_to_tk
from pytket.circuit.display import render_circuit_jupyter as draw

In [ ]:
def cbit_and(cbits, values):
    result = expr.bit_not(cbits[0]) if values[0] == 0 else expr.lift(cbits[0])
    for n in range(len(cbits)-1):
        result = expr.bit_and(result, expr.bit_not(cbits[n+1])) if values[n+1] == 0 else expr.bit_and(result, cbits[n+1])
    return result

def cbit_xor(cbits):
    result = expr.lift(cbits[0])
    for n in range(len(cbits)-1):
        result = expr.bit_xor(result, cbits[n+1])
    return result

In [ ]:
qreg = QuantumRegister(1, name="qreg")
qreg_setter = QuantumRegister(2, name="qsetter")
creg_A = ClassicalRegister(1, name="creg_A")
creg_B = ClassicalRegister(1, name="creg_B")
creg_C = ClassicalRegister(2, name="creg_C")
creg_D = ClassicalRegister(2, name="creg_D")
creg_E = ClassicalRegister(2, name="cintermediate_state")
creg_F = ClassicalRegister(5, name="creg_F")

qc = QuantumCircuit(qreg, qreg_setter, creg_A, creg_B, creg_C, creg_D, creg_E, creg_F)
qc.x(qreg_setter[1])

with qc.if_test((creg_A[0], 0)):
    qc.measure(qreg_setter[1], creg_B[0])

with qc.if_test(expr.lift(creg_A[0])):
    qc.measure(qreg_setter[1], creg_B[0])

with qc.if_test(expr.bit_not(creg_A[0])):
    qc.measure(qreg_setter[1], creg_B[0])

with qc.if_test(cbit_and([creg_A[0], creg_B[0]], [1, 0])) as _else:
    qc.measure(qreg[0], creg_C[0])
    
    qc.measure(qreg_setter[0], creg_E[:])
with _else:
    qc.measure(qreg[0], creg_C[0])
    
    qc.measure(qreg_setter[1], creg_E[:])

with qc.if_test(cbit_xor([creg_A[0], creg_B[0]])) as _else:
    qc.measure(qreg_setter[1], creg_C[0])
with _else:
    qc.measure(qreg_setter[0], creg_C[0])

with qc.if_test(cbit_and(creg_D, [1, 1])):
    qc.measure(qreg_setter[0], creg_A)

with qc.if_test(expr.bit_and(cbit_and(creg_C, [1, 1]), cbit_and(creg_D, [1,1]))) as _else:
    qc.measure(qreg_setter[0], creg_A)
with _else:
    qc.measure(qreg_setter[1], creg_A)

with qc.if_test(cbit_and(creg_C[:] + creg_D[:], [1, 1] + [1,1])) as _else:
    qc.measure(qreg_setter[0], creg_A)
with _else:
    qc.measure(qreg_setter[1], creg_A)

with qc.if_test(cbit_and(creg_F[:], [1]*creg_F.size)) as _else:
    qc.measure(qreg_setter[0], creg_A)

qc.draw("mpl")

In [ ]:
pm = PassManager([DecomposeIfElseOpsTask()])
qc_transpiled = qc.copy()
qc_transpiled = pm.run(qc_transpiled)
qc_transpiled.draw("mpl")

In [ ]:
circuit = qiskit_to_tk(qc_transpiled)

In [ ]:
draw(circuit)